In [2]:
import pandas as pd

def filter_consecutive_rows(df, columns_to_check):
    
    # Initialize a mask of rows to remove
    remove_mask = pd.Series(False, index=df.index)

    for col in columns_to_check:
        # Find groups of consecutive same values
        group_id = (df[col] != df[col].shift()).cumsum()
    
        # Group by those consecutive runs
        grouped = df.groupby(group_id)

        for _, idxs in grouped.groups.items():
            idxs = list(idxs)
            # Check if group has same value and length >= 3
            if len(idxs) >= 3:
                val = df.loc[idxs[0], col]
                if all(df.loc[i, col] == val for i in idxs):
                    # Mark all but the first for removal
                    remove_mask[idxs[1:]] = True

    # Apply mask to filter
    filtered_df = df[~remove_mask].reset_index(drop=True)
    
    # Return
    return filtered_df

In [3]:
# Sample DataFrame
df = pd.DataFrame({
    'A': [1, 1, 1, 2, 1, 1],
    'B': [5, 5, 5, 6, 7, 5],
    'C': [9, 9, 10, 11, 12, 9],
})
display(df)

filtered_df = filter_consecutive_rows(df, ['A', 'B', 'C'])
display(filtered_df)

,A,B,C
0,1,5,9
1,1,5,9
2,1,5,10
3,2,6,11
4,1,7,12
5,1,5,9


,A,B,C
0,1,5,9
1,2,6,11
2,1,7,12
3,1,5,9


# Swapping Two Column Values Based on a Third Column

In [4]:
df = pd.DataFrame({
    'A': [1, 2, 3, 4],
    'B': [10, 20, 30, 40],
    'Condition': ['yes', 'no', 'yes', 'no']
})
display(df)

mask = df['Condition'] == 'yes'

# Swap values using temporary variable
df.loc[mask, ['A', 'B']] = df.loc[mask, ['B', 'A']].values
display(df)

,A,B,Condition
0,1,10,yes
1,2,20,no
2,3,30,yes
3,4,40,no


,A,B,Condition
0,10,1,yes
1,2,20,no
2,30,3,yes
3,4,40,no


## Calculating Time Offsets, by Group

In [12]:
import numpy as np

# Sample data (your actual data may be larger or more complex)
df_samples = pd.DataFrame({
    'timestamp': [10.0, 10.05, 11.00, 11.30, 12.0],
    'group': ['A', 'A', 'A', 'A', 'B']
})

df_trials = pd.DataFrame({
    'start': [10.10, 11.10, 12.30],
    'end':   [10.20, 11.20, 12.40],
    'group': ['A', 'A', 'B']
})

display(df_samples)
display(df_trials)

,timestamp,group
0,10.00,A
1,10.05,A
2,11.00,A
3,11.30,A
4,12.00,B


,start,end,group
0,10.1,10.2,A
1,11.1,11.2,A
2,12.3,12.4,B


In [25]:
def CalculateOffsets(samples_df, sessions_df, group_colname, sample_timestamp_colname, session_start_colname, out_colname):
    
    results = []

    # Grouping by 'group'
    for group, sample_group in samples_df.groupby(group_colname):
        trial_group = sessions_df[sessions_df[group_colname] == group].sort_values(session_start_colname)
        trial_starts = trial_group[session_start_colname].values
        sample_timestamps = sample_group[sample_timestamp_colname].values

        # Initialize output
        deltas = []

        for ts in sample_timestamps:
            # Find first trial start AFTER this timestamp
            future_starts = trial_starts[trial_starts > ts]
            if len(future_starts) > 0:
                next_start = future_starts[0]
            else:
                next_start = trial_starts[-1]  # Fallback to latest trial start
            deltas.append(ts - next_start)

        # Append results for this group
        sample_group = sample_group.copy()
        sample_group[out_colname] = deltas
        results.append(sample_group)

    # Final DataFrame with the new column
    final_df = pd.concat(results).sort_index()
    return final_df[out_colname]

In [28]:
df_samples['offset3'] = CalculateOffsets(
    df_samples, 
    df_trials, 
    group_colname = 'group', 
    sample_timestamp_colname = 'timestamp',
    session_start_colname = 'start',
    out_colname = 'offset'
)

display(df_trials)
display(df_samples)

,start,end,group
0,10.1,10.2,A
1,11.1,11.2,A
2,12.3,12.4,B


,timestamp,group,offset2,offset3
0,10.00,A,-0.10,-0.10
1,10.05,A,-0.05,-0.05
2,11.00,A,-0.10,-0.10
3,11.30,A,0.20,0.20
4,12.00,B,-0.30,-0.30
